# Titanic 
- Kaggle Competition
- 목적 
    - Titanic을 푸는 것보단, Apache Spark에 적응하기
    - RDD보다 Dataframe API를 사용하기
- 실제론 Zeppelin을 사용하겠지만, Github에 올릴 때는 변환을 거쳐야 하므로(Zeppelin Notebook은 json으로 저장) jupyter notebook 사용 (커널 : toree)
    - 따라서 이 노트북에선 EDA를 시각적으로 표현하지 못했음

In [130]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col, udf, asc, sum, count, mean}

In [10]:
val spark = SparkSession
  .builder()
  .appName("Spark SQL basic example")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()

import spark.implicits._

spark = org.apache.spark.sql.SparkSession@442526a0


Spark local-1530447718546: Some(http://192.168.219.102:4040)

In [253]:
val trainData = "/Users/byeon/Dropbox/workspace/til/data/titanic-train.csv"
val testData = "/Users/byeon/Dropbox/workspace/til/data/titanic-test.csv"

trainData = /Users/byeon/Dropbox/workspace/til/data/titanic-train.csv
testData = /Users/byeon/Dropbox/workspace/til/data/titanic-test.csv


/Users/byeon/Dropbox/workspace/til/data/titanic-test.csv

In [52]:
val titanic = spark.read.option("header", "true").
    csv(trainData).
    withColumn("Survived", col("Survived").cast("Double")).
    withColumn("label", col("survived")).
    withColumn("Pclass", col("Pclass").cast("Double")).
    withColumn("SibSp", col("SibSp").cast("Double")).
    withColumn("Parch", col("Parch").cast("Double")).
    withColumn("Fare", col("Fare").cast("Double")).
    withColumn("Age", col("Age").cast("Int"))

titanic = [PassengerId: string, Survived: double ... 11 more fields]


[PassengerId: string, Survived: double ... 11 more fields]

In [103]:
titanic.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|label|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+-----+
|          1|     0.0|   3.0|Braund, Mr. Owen ...|  male| 22|  1.0|  0.0|       A/5 21171|   7.25| null|       S|  0.0|
|          2|     1.0|   1.0|Cumings, Mrs. Joh...|female| 38|  1.0|  0.0|        PC 17599|71.2833|  C85|       C|  1.0|
|          3|     1.0|   3.0|Heikkinen, Miss. ...|female| 26|  0.0|  0.0|STON/O2. 3101282|  7.925| null|       S|  1.0|
|          4|     1.0|   1.0|Futrelle, Mrs. Ja...|female| 35|  1.0|  0.0|          113803|   53.1| C123|       S|  1.0|
|          5|     0.0|   3.0|Allen, Mr. Willia...|  male| 35|  0.0|  0.0|          373450|   8.05| null|       S|  0.0|
+-----------+--------+------+-----------

In [104]:
titanic.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Pclass: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- label: double (nullable = true)



In [144]:
titanic.describe("Survived", "Age").show()

+-------+-------------------+------------------+
|summary|           Survived|               Age|
+-------+-------------------+------------------+
|  count|                891|               714|
|   mean| 0.3838383838383838|29.679271708683473|
| stddev|0.48659245426485753|14.536482769437564|
|    min|                0.0|                 0|
|    max|                1.0|                80|
+-------+-------------------+------------------+



In [109]:
titanic.describe("PClass", "Age", "SibSp").show()

+-------+------------------+------------------+------------------+
|summary|            PClass|               Age|             SibSp|
+-------+------------------+------------------+------------------+
|  count|               891|               714|               891|
|   mean| 2.308641975308642|29.679271708683473|0.5230078563411896|
| stddev|0.8360712409770491|14.536482769437564|1.1027434322934315|
|    min|               1.0|                 0|               0.0|
|    max|               3.0|                80|               8.0|
+-------+------------------+------------------+------------------+



In [55]:
titanic.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|     0.0|  549|
|     1.0|  342|
+--------+-----+



In [94]:
// 사망률
549.toFloat/891.toFloat

0.61616164

In [93]:
(549/891).toDouble

0.0

In [97]:
titanic.groupBy("Pclass", "Survived").count().orderBy("Pclass", "Survived").show()

+------+--------+-----+
|Pclass|Survived|count|
+------+--------+-----+
|   1.0|     0.0|   80|
|   1.0|     1.0|  136|
|   2.0|     0.0|   97|
|   2.0|     1.0|   87|
|   3.0|     0.0|  372|
|   3.0|     1.0|  119|
+------+--------+-----+



In [143]:
titanic.groupBy("Sex", "Survived").count().orderBy("Sex", "Survived").show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|female|     0.0|   81|
|female|     1.0|  233|
|  male|     0.0|  468|
|  male|     1.0|  109|
+------+--------+-----+



In [102]:
titanic.groupBy("Parch", "Survived").count().orderBy("Parch", "Survived").show()

+-----+--------+-----+
|Parch|Survived|count|
+-----+--------+-----+
|  0.0|     0.0|  445|
|  0.0|     1.0|  233|
|  1.0|     0.0|   53|
|  1.0|     1.0|   65|
|  2.0|     0.0|   40|
|  2.0|     1.0|   40|
|  3.0|     0.0|    2|
|  3.0|     1.0|    3|
|  4.0|     0.0|    4|
|  5.0|     0.0|    4|
|  5.0|     1.0|    1|
|  6.0|     0.0|    1|
+-----+--------+-----+



In [112]:
// zeppelin이나 spark notebook이면 시각화가 됨. jupyter notebook이라 안됨..
titanic.select("Fare").orderBy(asc("Fare")).map(row => row(0).asInstanceOf[Double]).collect()

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0125, 5.0, 6.2375, 6.4375, 6.45, 6.4958, 6.4958, 6.75, 6.75, 6.8583, 6.95, 6.975, 6.975, 7.0458, 7.05, 7.05, 7.05, 7.05, 7.05, 7.05, 7.05, 7.0542, 7.0542, 7.125, 7.125, 7.125, 7.125, 7.1417, 7.225, 7.225, 7.225, 7.225, 7.225, 7.225, 7.225, 7.225, 7.225, 7.225, 7.225, 7.225, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.2292, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.25, 7.3125, 7.4958, 7.4958, 7.4958, 7.5208, 7.55, 7.55, 7.55, 7.55, 7.6292, 7.65, 7.65, 7.65, 7.65, 7.725, 7.7292, 7.7333, 7.7333, 7.7333, 7.7333, 7.7375, 7.7375, 7.7417, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.75, 7.775, 7.775, 7.775, 7.775, 7.775, 7.775, 7.775, 7.775, 7.775, 7.775, 7.775, 7

In [123]:
titanic.groupBy("Sibsp").agg(sum("Survived"), count("Survived"), sum("Survived")/count("Survived")).orderBy("Sibsp").show()

+-----+-------------+---------------+---------------------------------+
|Sibsp|sum(Survived)|count(Survived)|(sum(Survived) / count(Survived))|
+-----+-------------+---------------+---------------------------------+
|  0.0|        210.0|            608|              0.34539473684210525|
|  1.0|        112.0|            209|               0.5358851674641149|
|  2.0|         13.0|             28|               0.4642857142857143|
|  3.0|          4.0|             16|                             0.25|
|  4.0|          3.0|             18|              0.16666666666666666|
|  5.0|          0.0|              5|                              0.0|
|  8.0|          0.0|              7|                              0.0|
+-----+-------------+---------------+---------------------------------+



In [127]:
val avgAge = titanic.select(mean("Age")).first()(0).asInstanceOf[Double]

avgAge = 29.679271708683473


29.679271708683473

In [140]:
val titanic2 = titanic.na.fill(avgAge, Seq("Age"))

titanic2 = [PassengerId: string, Survived: double ... 11 more fields]


[PassengerId: string, Survived: double ... 11 more fields]

In [141]:
titanic2.describe("Age").show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|               891|
|   mean|29.544332210998878|
| stddev|13.013778302447072|
|    min|                 0|
|    max|                80|
+-------+------------------+



- spark feature는 반드시 DOUBLE 타입

In [148]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

sqlContext = org.apache.spark.sql.SQLContext@213fe27e


org.apache.spark.sql.SQLContext@213fe27e

In [151]:
val toDouble = sqlContext.udf.register("toDouble", ((n: Int) => { n.toDouble }))

toDouble = UserDefinedFunction(<function1>,DoubleType,Some(List(IntegerType)))


UserDefinedFunction(<function1>,DoubleType,Some(List(IntegerType)))

In [157]:
val df = titanic2.drop("Name").drop("cabin").drop("Ticket").drop("Embarked").
                                   withColumn("Survived", toDouble(titanic2("Survived")))

df = [PassengerId: string, Survived: double ... 7 more fields]


[PassengerId: string, Survived: double ... 7 more fields]

In [158]:
df.printSchema

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Pclass: double (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- label: double (nullable = true)



## 모델링

In [159]:
import org.apache.spark.ml.feature.StringIndexer

In [161]:
// StringIndexer를 활용해 String을 index로 바꿈
val sexIndexer = new StringIndexer().setInputCol("Sex").setOutputCol("SexIndex")

sexIndexer = strIdx_36193a26a9e4


strIdx_36193a26a9e4

In [162]:
var tempModel = sexIndexer.fit(df)

tempModel = strIdx_36193a26a9e4


strIdx_36193a26a9e4

In [163]:
tempModel.transform(df).select("Sex", "SexIndex").show(5)

+------+--------+
|   Sex|SexIndex|
+------+--------+
|  male|     0.0|
|female|     1.0|
|female|     1.0|
|female|     1.0|
|  male|     0.0|
+------+--------+
only showing top 5 rows



In [164]:
import org.apache.spark.ml.feature.Bucketizer
// Category화(Bucket화)

In [171]:
val fareSplits = Array(0.0, 50.0, 100.0, 150.0, 200.0, Double.PositiveInfinity)
val fareBucket = new Bucketizer().setInputCol("Fare").setOutputCol("FareBucket").setSplits(fareSplits)

fareSplits = Array(0.0, 50.0, 100.0, 150.0, 200.0, Infinity)
fareBucket = bucketizer_20b07578fd17


bucketizer_20b07578fd17

In [176]:
fareBucket.transform(df).select("Fare", "FareBucket").show(20)

+-------+----------+
|   Fare|FareBucket|
+-------+----------+
|   7.25|       0.0|
|71.2833|       1.0|
|  7.925|       0.0|
|   53.1|       1.0|
|   8.05|       0.0|
| 8.4583|       0.0|
|51.8625|       1.0|
| 21.075|       0.0|
|11.1333|       0.0|
|30.0708|       0.0|
|   16.7|       0.0|
|  26.55|       0.0|
|   8.05|       0.0|
| 31.275|       0.0|
| 7.8542|       0.0|
|   16.0|       0.0|
| 29.125|       0.0|
|   13.0|       0.0|
|   18.0|       0.0|
|  7.225|       0.0|
+-------+----------+
only showing top 20 rows



In [189]:
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().
    setInputCols(Array("Pclass", "SexIndex", "Age", "SibSp", "Parch", "FareBucket")).
    setOutputCol("tempFeatures")

assembler = vecAssembler_18ed537fd910


lastException: Throwable = null


vecAssembler_18ed537fd910

In [178]:
import org.apache.spark.ml.feature.Normalizer


In [190]:
val normalizer = new Normalizer().setInputCol("tempFeatures").setOutputCol("features")

normalizer = normalizer_fb823ceff85e


normalizer_fb823ceff85e

In [181]:
import org.apache.spark.ml.classification.LogisticRegression

In [191]:
val logreg = new LogisticRegression().setMaxIter(10)
logreg.setLabelCol("Survived")

logreg = logreg_9380ab7308c9


logreg_9380ab7308c9

In [183]:
import org.apache.spark.ml.Pipeline

In [192]:
val pipeline = new Pipeline().setStages(Array(fareBucket, sexIndexer, assembler, normalizer, logreg))

pipeline = pipeline_a63f0926dd04


pipeline_a63f0926dd04

In [193]:
val splits = df.randomSplit(Array(0.7, 0.3), seed=9L)
val train = splits(0).cache()
val test = splits(1).cache()

splits = Array([PassengerId: string, Survived: double ... 7 more fields], [PassengerId: string, Survived: double ... 7 more fields])
train = [PassengerId: string, Survived: double ... 7 more fields]
test = [PassengerId: string, Survived: double ... 7 more fields]


[PassengerId: string, Survived: double ... 7 more fields]

In [194]:
val model = pipeline.fit(train)
val result = model.transform(test)

model = pipeline_a63f0926dd04
result = [PassengerId: string, Survived: double ... 14 more fields]


[PassengerId: string, Survived: double ... 14 more fields]

In [197]:
result.select("PassengerId", "Survived").show(5)

+-----------+--------+
|PassengerId|Survived|
+-----------+--------+
|          1|     0.0|
|        104|     0.0|
|        106|     0.0|
|        107|     1.0|
|        109|     0.0|
+-----------+--------+
only showing top 5 rows



In [204]:
result.printSchema

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Pclass: double (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- label: double (nullable = true)
 |-- FareBucket: double (nullable = true)
 |-- SexIndex: double (nullable = false)
 |-- tempFeatures: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [232]:
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics


In [235]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [243]:
val predictionAndLabel = result.select("prediction", "Survived").
    map(row => (row.get(0).asInstanceOf[Double]))

predictionAndLabel = [prediction: double, Survived: double]


[prediction: double, Survived: double]

In [244]:
predictionAndLabel.show(3)

+----------+--------+
|prediction|Survived|
+----------+--------+
|       0.0|     0.0|
|       0.0|     0.0|
|       0.0|     0.0|
+----------+--------+
only showing top 3 rows



In [236]:
val metrics = new BinaryClassificationEvaluator()

metrics = binEval_d790703769c5


binEval_d790703769c5

In [245]:
metrics.setRawPredictionCol("prediction").setLabelCol("Survived")

binEval_d790703769c5

In [249]:
metrics.setMetricName("areaUnderROC")
println(metrics.evaluate(predictionAndLabel))

0.7123397435897436


In [255]:
var testDf = spark.read.option("header", "true").
    csv(testData).
    toDF().
    cache()

testDf = [PassengerId: string, Pclass: string ... 9 more fields]


[PassengerId: string, Pclass: string ... 9 more fields]

In [258]:
testDf.describe("Fare").show()

+-------+------------------+
|summary|              Fare|
+-------+------------------+
|  count|               417|
|   mean|  35.6271884892086|
| stddev|55.907576179973844|
|    min|                 0|
|    max|              93.5|
+-------+------------------+



In [ ]:
// ToDo : testDf 제출 자료로 만들기